In [39]:
try:
    import evidently
except: 
    %pip install git+https://github.com/evidentlyai/evidently.git

In [40]:
import pandas as pd 

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

In [41]:
%pip install xlrd

In [42]:
from scipy.io import arff

# Load the data
data = arff.loadarff('/Users/hirushau/Code/Model-Monitoring/datasets/real_datasets/kddcup99.arff')
df = pd.DataFrame(data[0])

# Assuming 'df' is your DataFrame
for col in df.columns:
    df[col] = df[col].apply(lambda x: x.decode() if isinstance(x, bytes) else x)


# Now you can view your data
print(df.head())

   duration protocol_type   service flag  src_bytes  dst_bytes land  \
0       0.0           udp  domain_u   SF       46.0       46.0    0   
1       0.0          icmp     ecr_i   SF      520.0        0.0    0   
2       0.0          icmp     ecr_i   SF     1032.0        0.0    0   
3       0.0           tcp   private   S0        0.0        0.0    0   
4       0.0           tcp   private   S0        0.0        0.0    0   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0             0.0     0.0  0.0  ...               255.0   
1             0.0     0.0  0.0  ...               255.0   
2             0.0     0.0  0.0  ...               255.0   
3             0.0     0.0  0.0  ...                 1.0   
4             0.0     0.0  0.0  ...                11.0   

  dst_host_same_srv_rate  dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                   1.00                    0.00                          0.0   
1                   1.00                    0.00        

In [5]:
# load data 
df = pd.read_excel("datasets/real_datasets/default of credit card clients.xls")
df = df.drop(df.index[0])
df = df.rename(columns={"Unnamed: 0": "ID"})
df.head()
# print(df.columns)

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [43]:
# Split your data into reference and current datasets
reference_df = df.sample(frac=0.3, random_state=1)
current_df = df.drop(reference_df.index)

In [10]:
from evidently import ColumnMapping

column_mapping = ColumnMapping()

# Define categorical features
column_mapping.categorical_features = []

# Define numerical features
# column_mapping.numerical_features = ['attribute0','attribute1','attribute2','attribute3','attribute4','attribute5',\
#     'attribute6','attribute7','attribute8','attribute9' ]
column_mapping.numerical_features = ['attribute0','attribute1']


In [8]:
from evidently import ColumnMapping

column_mapping = ColumnMapping()

# Define categorical features
column_mapping.categorical_features = []

# Define numerical features
column_mapping.numerical_features = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',\
       'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20',\
       'X21', 'X22', 'X23']

In [44]:
from evidently import ColumnMapping

column_mapping = ColumnMapping()

# Specify the target column
column_mapping.target = 'class'
# Specify numerical features
column_mapping.numerical_features = ['duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot',\
        'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', \
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',\
        'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', \
       'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', \
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', \
              'dst_host_srv_rerror_rate']

# Specify categorical features
column_mapping.categorical_features = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login']


In [45]:
# Create a report with DataDriftPreset
data_drift_report = Report(metrics=[DataDriftPreset(stattest='psi',stattest_threshold=0.01)])
data_drift_report.run(reference_data=reference_df, current_data=current_df, column_mapping=column_mapping)

In [46]:
# Visualize the report
data_drift_report.save_html("kddcup99_report.html")